# GHCN Monthly Average Station Data

* Warning: this code does the ingestion as well. Run with caution.
* This notebook was once used in "production". Now it serves only as an inspiration for the GHCNM module

In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, addcyclic, shiftgrid
import pymongo
from pprint import pprint
from datetime import datetime, timedelta, date
import pandas as pd
from sklearn.decomposition import PCA
import sklearn.linear_model as skl_lm
import gdal as gdl
import matplotlib.mlab as ml
import cartopy.crs as ccrs
import plotly.graph_objs as go
import plotly.offline as py
py.init_notebook_mode(connected=True) # for live plot
pd.set_option('display.notebook_repr_html', False)
%matplotlib inline
plt.style.use('seaborn-white')

In [2]:
mongo_host_local = 'mongodb://localhost:27017/'
mg = pymongo.MongoClient(mongo_host_local)
db = mg.GHCN
db.collection_names()

['data', 'system.indexes', 'stations']

In [3]:
mg.database_names()

['ECMWF', 'local', 'GHCN']

# Get Target Variables

### GHCN Monthly Data

* ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/v3/
* "QCA" files represent the quality controlled adjusted data.

## Country Code

In [4]:
country_df = pd.read_fwf('/home/dmasson/data/ghcnm/country-codes',
                     colspecs=[[0,3], [4, 50]],
                     header=None,
                     names=['country_id', 'country'])
country_df.head()

   country_id       country
0         101       ALGERIA
1         102        ANGOLA
2         103         BENIN
3         104      BOTSWANA
4         105  BURKINA FASO

## Station Metadata

".qca.inv" is (?) the metadata containing lon, lat, etc:

       Variable          Columns      Type
       --------          -------      ----

       ID                 1-11        Integer
       LATITUDE          13-20        Real
       LONGITUDE         22-30        Real
       STNELEV           32-37        Real          (elevation in meters)
       NAME              39-68        Character
       GRELEV            70-73        Integer
       POPCLS            74-74        Character
       POPSIZ            75-79        Integer
       TOPO              80-81        Character
       STVEG             82-83        Character
       STLOC             84-85        Character
       OCNDIS            86-87        Integer
       AIRSTN            88-88        Character
       TOWNDIS           89-90        Integer
       GRVEG             91-106       Character
       POPCSS            107-107      Character
       

In [8]:
# Station metadata
sta_df = pd.read_fwf('/home/dmasson/data/ghcnm/ghcnm.tavg.v3.3.0.20180311.qca.inv',
                     colspecs=[[0,11], [0,3],[3,8],[13, 20], [24, 30], [31,37], [38,69], [90, 106],[106,107]],
                     header=None,
                     #nrows=20,
                     names=['station_id','country_id','wmo_id', 
                            'lat', 'lon', 'elev', 'name', 'landcover', 'popclass'])
sta_df = pd.merge(sta_df, country_df, on='country_id')
sta_df.head()

    station_id  country_id  wmo_id    lat   lon   elev          name  \
0  10160355000         101   60355  36.93  6.95    7.0        SKIKDA   
1  10160360000         101   60360  36.83  7.82    4.0        ANNABA   
2  10160390000         101   60390  36.72  3.25   25.0  DAR-EL-BEIDA   
3  10160395001         101   60395  36.52  4.18  942.0  FT. NATIONAL   
4  10160400001         101   60400  36.80  5.10  230.0    CAP CARBON   

        landcover popclass  country  
0  WARM DECIDUOUS        C  ALGERIA  
1      WARM CROPS        C  ALGERIA  
2      WARM CROPS        C  ALGERIA  
3      WARM CROPS        A  ALGERIA  
4           WATER        A  ALGERIA  

In [9]:
def upsertStation(i):
    newdoc = dict({'station_id': int(sta_df.station_id[i]), 
                   'loc': {'type': 'Point', 'coordinates': [float(sta_df.lon[i]), float(sta_df.lat[i])]}, 
                   'country': sta_df.country[i], 
                   'country_id': int(sta_df.country_id[i]), 
                   'wmo_id': int(sta_df.wmo_id[i]), 
                   'elev': sta_df.elev[i], 
                   'name': sta_df.name[i], 
                   'landcover': sta_df.landcover[i], 'popclass': sta_df.popclass[i]})
    db.stations.update_one(filter={"station_id": newdoc['station_id']}, update=dict({'$set': newdoc}), upsert=True)


In [10]:
void = list(map(upsertStation, np.arange(sta_df.shape[0])))

In [11]:
db.stations.count()

7280

In [59]:
#db.stations.drop()

In [12]:
db.stations.find_one()

{'_id': ObjectId('5abde7e773d02a7f26ab4dd7'),
 'country': 'ALGERIA',
 'country_id': 101,
 'elev': 7.0,
 'landcover': 'WARM DECIDUOUS',
 'loc': {'coordinates': [6.95, 36.93], 'type': 'Point'},
 'name': 'SKIKDA',
 'popclass': 'C',
 'station_id': 10160355000,
 'wmo_id': 60355}

In [74]:
def doStationIndexing():
    # Add indexes
    # Warning: geospatial index require -180, +180 longitudes !!
    db.stations.create_index([("station_id", pymongo.ASCENDING)])
    db.stations.create_index([("loc", pymongo.GEOSPHERE)])
    
#doStationIndexing()

In [13]:
db.stations.index_information()

{'_id_': {'key': [('_id', 1)], 'ns': 'GHCN.stations', 'v': 1},
 'loc_2dsphere': {'2dsphereIndexVersion': 3,
  'key': [('loc', '2dsphere')],
  'ns': 'GHCN.stations',
  'v': 1},
 'station_id_1': {'key': [('station_id', 1)], 'ns': 'GHCN.stations', 'v': 1}}

## Station Observations

* ".qca.dat" contains the observation:

          Variable          Columns      Type
          --------          -------      ----

          ID                 1-11        Integer
          YEAR              12-15        Integer
          ELEMENT           16-19        Character  (i.e. variable name)
          VALUE1            20-24        Integer    (1/100 th of degree Celsius, MISSING=-9999)
          DMFLAG1           25-25        Character
          QCFLAG1           26-26        Character
          DSFLAG1           27-27        Character
            .                 .             .
            .                 .             .
            .                 .             .
          VALUE12          108-112       Integer
          DMFLAG12         113-113       Character
          QCFLAG12         114-114       Character
          DSFLAG12         115-115       Character

The Temparature data is given in hundredths of °C

In [17]:
# Station data
dat_df = pd.read_fwf('/home/dmasson/data/ghcnm/ghcnm.tavg.v3.3.0.20180311.qca.dat',
                     na_values='-9999',
                     colspecs=[[0,11], [11, 15], [15,19], 
                               [19,24], [27,32],[35,40],[43,48],[51,56],
                               [59,64],[67,72],[75,80],[83,88],[91,96],[99,104],[107,112]],
                     header=None,
                     #nrows=20,
                     names=['station_id', 'year', 'variable',
                            '1','2','3','4','5','6','7','8','9','10','11','12'])
# Convertion to °C
dat_df['1'] = dat_df['1']/100
dat_df['2'] = dat_df['2']/100 
dat_df['3'] = dat_df['3']/100 
dat_df['4'] = dat_df['4']/100 
dat_df['5'] = dat_df['5']/100 
dat_df['6'] = dat_df['6']/100 
dat_df['7'] = dat_df['7']/100 
dat_df['8'] = dat_df['8']/100 
dat_df['9'] = dat_df['9']/100 
dat_df['10'] = dat_df['10']/100 
dat_df['11'] = dat_df['11']/100 
dat_df['12'] = dat_df['12']/100 

dat_df.head()

    station_id  year variable      1      2      3      4      5      6  \
0  10160355000  1878     TAVG   9.61  10.21  11.81  16.81  20.51  23.11   
1  10160355000  1879     TAVG  12.51  12.41  12.91  16.21  16.31  23.41   
2  10160355000  1880     TAVG  10.31  11.81  13.11  15.91  17.81  21.41   
3  10160355000  1931     TAVG    NaN  10.41    NaN    NaN  19.21  24.61   
4  10160355000  1932     TAVG  10.81  10.51    NaN  14.91  19.11    NaN   

       7      8      9     10     11     12  
0  25.61  27.51  23.91    NaN  14.41  12.21  
1  24.71  25.81  23.11  18.21  15.21   9.71  
2  26.51  26.41  23.61  21.31  16.41  13.61  
3    NaN  26.71  22.31  20.01  16.21  11.31  
4  23.61    NaN  25.11    NaN    NaN    NaN  

In my opinion, it is possible to insert the table above as it is in MongoDB. We anyway need to group by month later in the analysis

In [18]:
db.data.insert_many(dat_df.to_dict('records'))


In [19]:
db.data.count()

470883

In [15]:
#db.data.drop()

In [20]:
db.data.find_one()

{'1': 9.61,
 '10': nan,
 '11': 14.41,
 '12': 12.21,
 '2': 10.21,
 '3': 11.81,
 '4': 16.81,
 '5': 20.51,
 '6': 23.11,
 '7': 25.61,
 '8': 27.51,
 '9': 23.91,
 '_id': ObjectId('5abe7e508cb6b80adfb07707'),
 'station_id': 10160355000,
 'variable': 'TAVG',
 'year': 1878}

In [21]:
def doDataIndexing():
    # Add indexes
    # Warning: geospatial index require -180, +180 longitudes !!
    db.data.create_index([("station_id", pymongo.ASCENDING)])
    db.data.create_index([("year", pymongo.DESCENDING)])

#doDataIndexing()

In [22]:
db.data.index_information()

{'_id_': {'key': [('_id', 1)], 'ns': 'GHCN.data', 'v': 1},
 'station_id_1': {'key': [('station_id', 1)], 'ns': 'GHCN.data', 'v': 1},
 'year_-1': {'key': [('year', -1)], 'ns': 'GHCN.data', 'v': 1}}